# Assignment 1 - Propagation Settings

In [ ]:
''' 
Copyright (c) 2010-2020, Delft University of Technology
All rigths reserved

This file is part of the Tudat. Redistribution and use in source and 
binary forms, with or without modification, are permitted exclusively
under the terms of the Modified BSD license. You should have received
a copy of the license with this file. If not, please or visit:
http://tudat.tudelft.nl/LICENSE.
'''

import numpy as np
from tudatpy import elements
from tudatpy.io import save2txt
from tudatpy.kernel import constants
from tudatpy.kernel.interface import spice_interface
from tudatpy.kernel.simulation import environment_setup
from tudatpy.kernel.simulation import propagation_setup
from matplotlib import pyplot as plt

In [ ]:
# student number: 1244779 --> 1244ABC
A = XXXX
B = XXXX
C = XXXX

simulation_start_epoch = 33.15 * constants.JULIAN_YEAR + A * 7.0 * constants.JULIAN_DAY + \
                            B * constants.JULIAN_DAY + C * constants.JULIAN_DAY / 24.0
simulation_end_epoch = simulation_start_epoch + 344.0 * constants.JULIAN_DAY / 24.0

## Create Environment and Vehicle

In [ ]:
###########################################################################
# CREATE ENVIRONMENT ######################################################
###########################################################################

# Load spice kernels.
spice_interface.load_standard_kernels()

# Create body objects.
bodies_to_create = ["Ganymede", "Jupiter"]
global_frame_origin = "SSB"
global_frame_orientation = "ECLIPJ2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation)    

# Add Ganymede exponential atmosphere 
density_scale_height = 40.0E3
density_at_zero_altitude = 2.0E-9
body_settings.get( "Ganymede" ).atmosphere_settings = environment_setup.atmosphere.exponential( 
        density_scale_height, density_at_zero_altitude)

bodies = environment_setup.create_system_of_bodies(body_settings)

###########################################################################
# CREATE VEHICLE ##########################################################
###########################################################################

# Create vehicle object
bodies.create_empty_body( "JUICE" )

# Set mass of vehicle
bodies.get_body( "JUICE" ).set_constant_mass(2000.0)

# Create aerodynamic coefficients interface
reference_area = 100.0
drag_coefficient = 1.2
aero_coefficient_settings = environment_setup.aerodynamic_coefficients.constant(
        reference_area,[drag_coefficient,0,0] )
environment_setup.add_aerodynamic_coefficient_interface(
        bodies, "JUICE", aero_coefficient_settings );

## Propagate Dynamics for various cases

In [ ]:
cases = ['unperturbed', 'case_i', 'case_ii']

"""
unperturbed: Ganymede PM

case_i: Ganymede PM, Jupiter SH D/O 4/0

case_ii: Ganymede PM, Ganymede aerodynamic
"""

simulation_results_dict = dict()
dependent_variables_dict = dict()
for case in cases:    
    ###########################################################################
    # CREATE ACCELERATIONS ####################################################
    ###########################################################################

    # Define bodies that are propagated.
    bodies_to_propagate = ["JUICE"]

    # Define central bodies.
    central_bodies = ["Ganymede"]

    # Define accelerations acting on vehicle.
    if case == 'unperturbed':
        acceleration_settings_on_vehicle = dict(
            Ganymede = XXXX
        )
    if case == 'case_i':
        acceleration_settings_on_vehicle = dict(
            Ganymede = XXXX,
            Jupiter = XXXX
        )
    if case == 'case_ii':
        acceleration_settings_on_vehicle = dict(
            Ganymede = XXXX
        )

    # Create global accelerations dictionary.
    acceleration_settings = {"JUICE": acceleration_settings_on_vehicle}

    # Create acceleration models.
    acceleration_models = propagation_setup.create_acceleration_models(
            bodies, acceleration_settings, bodies_to_propagate, central_bodies)


    ###########################################################################
    # CREATE PROPAGATION SETTINGS #############################################
    ###########################################################################

    # Define initial state.
    system_initial_state = spice_interface.get_body_cartesian_state_at_epoch(
       target_body_name="JUICE",
       observer_body_name="Ganymede",
       reference_frame_name="ECLIPJ2000",
       aberration_corrections="NONE",
       ephemeris_time= simulation_start_epoch )

    # Save magnitude of perturbations for both cases
    if case == 'unperturbed':
        dependent_variables_to_save = [ ]
    if case == 'case_i':
        dependent_variables_to_save = [ 
            propagation_setup.dependent_variable.XXXX
        ]
    if case == 'case_ii':
        dependent_variables_to_save = [ 
            propagation_setup.dependent_variable.XXXX
        ]

    # Create propagation settings.
    propagator_settings = propagation_setup.propagator.translational(
        central_bodies,
        acceleration_models,
        bodies_to_propagate,
        system_initial_state,
        simulation_end_epoch,
        output_variables = dependent_variables_to_save
    )

    # Create numerical integrator settings.
    fixed_step_size = 10.0
    integrator_settings = propagation_setup.integrator.runge_kutta_4(
        simulation_start_epoch,
        fixed_step_size
    )

    ###########################################################################
    # PROPAGATE ORBIT #########################################################
    ###########################################################################

    # Create simulation object and propagate dynamics.
    dynamics_simulator = propagation_setup.SingleArcDynamicsSimulator(
        bodies, integrator_settings, propagator_settings)
    
    simulation_results_dict[case] = dynamics_simulator.state_history
    dependent_variables_dict[case] = dynamics_simulator.dependent_variable_history

    ###########################################################################
    # PRINT FINAL PROPAGATION TIME AND STATE ##################################
    ###########################################################################

    final_time_step=list(simulation_results_dict[case].keys())[-1]
    first_time_step=list(simulation_results_dict[case].keys())[0]

    print(
        f"""
    JUICE Propagation Results of {case}.

    Final propagation time of JUICE [s]: {simulation_end_epoch}
    Final Cartesian state of JUICE is [m]: \n{
        simulation_results_dict[case][final_time_step][:]}

        """
    )

    ###########################################################################
    # SAVE RESULTS ############################################################
    ###########################################################################
    
#     save2txt(solution=simulation_result,
#              filename="JUICEPropagationHistory_Q4_" + case + ".dat",
#              directory="./",  # default = "./" 
#              column_names=None,  # default = None 
#              )
    
    

## Pre-process Results

In [ ]:
simulation_result_unperturbed = simulation_results_dict[ 'unperturbed']
simulation_result_i = simulation_results_dict[ 'case_i' ]
simulation_result_ii = simulation_results_dict[ 'case_ii' ]

dependent_variables_unperturbed = dependent_variables_dict[ 'unperturbed' ]
dependent_variables_i = dependent_variables_dict[ 'case_i' ]
dependent_variables_ii = dependent_variables_dict[ 'case_ii' ]

difference_in_cartesian_position = XXXX

## Plot Results